<a href="https://colab.research.google.com/github/camiPadilla/SIS420CP/blob/main/Primer_Parcial_120.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
#Implementar librerias
import os
#computacion vectorial y cientifica
import numpy as np
#libreria para graficar
from matplotlib import pyplot

from sklearn.preprocessing import LabelEncoder
from mpl_toolkits.mplot3d import Axes3D #Para graficar elementos 3D

#llama a mtplotlib a embeber graficas dentro de los cuadernillos
%matplotlib inline

In [6]:
from sklearn.model_selection import train_test_split
import pandas as pd

In [7]:
#Carga de dataset
data = pd.read_csv('PS_20174392719_1491204439457_log.csv', header=0, sep=',')

In [8]:
print(data)

       step      type     amount     nameOrig  oldbalanceOrg  newbalanceOrig  \
0         1   PAYMENT    9839.64  C1231006815       170136.0       160296.36   
1         1   PAYMENT    1864.28  C1666544295        21249.0        19384.72   
2         1  TRANSFER     181.00  C1305486145          181.0            0.00   
3         1  CASH_OUT     181.00   C840083671          181.0            0.00   
4         1   PAYMENT   11668.14  C2048537720        41554.0        29885.86   
...     ...       ...        ...          ...            ...             ...   
83556    10   PAYMENT    7705.70  C1834114901        96490.0        88784.30   
83557    10  CASH_OUT  319045.01  C1964329082        56471.0            0.00   
83558    10   CASH_IN  249169.96  C1421944154         3481.0       252650.96   
83559    10  CASH_OUT  244279.64   C722886752        29968.0            0.00   
83560    10  CASH_OUT  145014.63    C60491101            NaN             NaN   

          nameDest  oldbalanceDest  new

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83561 entries, 0 to 83560
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   step            83561 non-null  int64  
 1   type            83561 non-null  object 
 2   amount          83561 non-null  float64
 3   nameOrig        83561 non-null  object 
 4   oldbalanceOrg   83560 non-null  float64
 5   newbalanceOrig  83560 non-null  float64
 6   nameDest        83560 non-null  object 
 7   oldbalanceDest  83560 non-null  float64
 8   newbalanceDest  83560 non-null  float64
 9   isFraud         83560 non-null  float64
 10  isFlaggedFraud  83560 non-null  float64
dtypes: float64(7), int64(1), object(3)
memory usage: 7.0+ MB


In [10]:
#NOTA: Las transacciones que se detectan como fraude se cancelan,
#por lo que para la detección de fraude no se deben utilizar estas columnas:
#(oldbalanceOrg, newbalanceOrig, oldbalanceDest, newbalanceDest).

#Dado esos detalles , eliminamos columnas
columna_eliminar = ['oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']
data.drop(columna_eliminar, axis=1, inplace=True)
# axis igual a 1 para columnas, igual a 0 para filas
# inplace igual a true borra la columna en el dataframe original

In [11]:
# Convertimos las columnas categoricas en columnas numericas
columnas_categoricas = data.select_dtypes(include=['object']).columns

In [14]:
#procesamiento de frases
for columna in columnas_categoricas:
  le = LabelEncoder()
  data[columna] = le.fit_transform(data[columna])
data[columnas_categoricas] = data[columnas_categoricas].astype('int64')

In [15]:
# observamos si nos faltan datos
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83561 entries, 0 to 83560
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   step            83561 non-null  int64  
 1   type            83561 non-null  int64  
 2   amount          83561 non-null  float64
 3   nameOrig        83561 non-null  int64  
 4   nameDest        83561 non-null  int64  
 5   isFraud         83560 non-null  float64
 6   isFlaggedFraud  83560 non-null  float64
dtypes: float64(3), int64(4)
memory usage: 4.5 MB


In [16]:
# Leemos el dataset y seleccionamos las columnas para X y y
X = data.iloc[:, :6]
y = data.iloc[:, 6]
m = y.size

In [17]:
print(X)
print('-'*15)
print(y)

       step  type     amount  nameOrig  nameDest  isFraud
0         1     3    9839.64      9871     27341      0.0
1         1     3    1864.28     28654     28452      0.0
2         1     4     181.00     13026      8056      1.0
3         1     1     181.00     76619      7157      1.0
4         1     3   11668.14     45087     14353      0.0
...     ...   ...        ...       ...       ...      ...
83556    10     3    7705.70     35927     14101      0.0
83557    10     1  319045.01     41374      8818      0.0
83558    10     0  249169.96     18119      9324      0.0
83559    10     1  244279.64     71379      2668      0.0
83560    10     1  145014.63     66289     43957      NaN

[83561 rows x 6 columns]
---------------
0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
83556    0.0
83557    0.0
83558    0.0
83559    0.0
83560    NaN
Name: isFlaggedFraud, Length: 83561, dtype: float64
